In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_model_optimization as tfmot
from tensorflow.keras.models import load_model
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical

# 1. Load Fashion MNIST dataset
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# Normalize and reshape to (batch, height, width, channels)
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0
X_train = np.expand_dims(X_train, axis=-1)  # shape: (60000, 28, 28, 1)
X_test = np.expand_dims(X_test, axis=-1)

# One-hot encode labels
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# 2. Load baseline model
baseline_model = load_model('baseline_model2.h5')
print("✅ Baseline model loaded")

# 3. Apply weight clustering
cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

clustering_params = {
    'number_of_clusters': 16,
    'cluster_centroids_init': CentroidInitialization.KMEANS_PLUS_PLUS
}

clustered_model = cluster_weights(baseline_model, **clustering_params)
print("✅ Weight clustering applied")

# 4. Compile clustered model
opt = tf.keras.optimizers.Adam(learning_rate=1e-5)
clustered_model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    optimizer=opt,
    metrics=['accuracy']
)
print("✅ Clustered model compiled and ready for training")

# Optional: train clustered model (example)
# epochs = 5
# clustered_model.fit(X_train, y_train, batch_size=128, epochs=epochs, validation_split=0.1)


✅ Baseline model loaded


✅ Weight clustering applied
✅ Clustered model compiled and ready for training


In [2]:
print(clustered_model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cluster_conv2d (ClusterWei  (None, 26, 26, 32)        624       
 ghts)                                                           
                                                                 
 cluster_max_pooling2d (Clu  (None, 13, 13, 32)        0         
 sterWeights)                                                    
                                                                 
 cluster_conv2d_1 (ClusterW  (None, 11, 11, 64)        36944     
 eights)                                                         
                                                                 
 cluster_max_pooling2d_1 (C  (None, 5, 5, 64)          0         
 lusterWeights)                                                  
                                                                 
 cluster_flatten (ClusterWe  (None, 1600)              0

In [3]:
clustered_model.fit(
X_train,
y_train,
batch_size=128,
epochs=2,
validation_split=0.1)

Epoch 1/2


/Users/oscarpatrikminj/Documents/IITR/FMNIST/venv/lib/python3.10/site-packages/keras/src/backend.py:5562: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


422/422 [==============================] - 19s 44ms/step - loss: 0.1385 - accuracy: 0.9515 - val_loss: 0.2217 - val_accuracy: 0.9187
Epoch 2/2
422/422 [==============================] - 17s 40ms/step - loss: 0.1289 - accuracy: 0.9543 - val_loss: 0.2181 - val_accuracy: 0.9213


In [5]:
import tensorflow as tf
import tensorflow_model_optimization as tfmot
import pathlib

# Strip clustering wrappers to get final clustered model
model_for_export = tfmot.clustering.keras.strip_clustering(clustered_model)

# Convert the stripped model to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
tflite_model_clustered = converter.convert()

# Define your absolute path
tflite_models_dir = pathlib.Path("/Users/oscarpatrikminj/Documents/IITR/FMNIST/tflite_models")

# Create directory if not exists
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# Save the clustered TFLite model
tflite_model_file = tflite_models_dir / "model_clustered.tflite"
tflite_model_file.write_bytes(tflite_model_clustered)

print(f"✅ Clustered TFLite model saved at: {tflite_model_file}")


INFO:tensorflow:Assets written to: /var/folders/bs/x0lj933d1hv0py0d4w2ypdp40000gn/T/tmp857v65tg/assets


INFO:tensorflow:Assets written to: /var/folders/bs/x0lj933d1hv0py0d4w2ypdp40000gn/T/tmp857v65tg/assets


✅ Clustered TFLite model saved at: /Users/oscarpatrikminj/Documents/IITR/FMNIST/tflite_models/model_clustered.tflite


2025-06-05 18:49:42.273797: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-06-05 18:49:42.273812: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-06-05 18:49:42.274002: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/bs/x0lj933d1hv0py0d4w2ypdp40000gn/T/tmp857v65tg
2025-06-05 18:49:42.274881: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-06-05 18:49:42.274888: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /var/folders/bs/x0lj933d1hv0py0d4w2ypdp40000gn/T/tmp857v65tg
2025-06-05 18:49:42.277634: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2025-06-05 18:49:42.299152: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: /var/folders/bs/x0lj933d1hv0py0d4w2ypdp40000gn/T/tmp857v65tg
2025-06-

In [6]:
import numpy as np
import tensorflow as tf
import tempfile
import os
import zipfile
from pathlib import Path

# Function to get gzipped model size
def get_gzipped_model(file_path):
    _, zipped_file = tempfile.mkstemp('.zip')
    with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
        f.write(file_path, arcname=Path(file_path).name)
    size = os.path.getsize(zipped_file)
    os.remove(zipped_file)
    return size

# Load the clustered TFLite model
tflite_model_file = 'tflite_models/model_clustered.tflite'
interpreter = tf.lite.Interpreter(model_path=tflite_model_file)
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

pred_list = []

for image in X_test:
    input_data = np.array(image, dtype=np.float32)
    input_data = input_data.reshape(1, input_data.shape[0], input_data.shape[1], 1)
    interpreter.set_tensor(input_index, input_data)
    interpreter.invoke()
    prediction = interpreter.get_tensor(output_index)
    prediction = np.argmax(prediction)
    pred_list.append(prediction)

accurate_count = 0
for idx in range(len(pred_list)):
    if pred_list[idx] == np.argmax(y_test[idx]):
        accurate_count += 1

accuracy = accurate_count / len(pred_list)
print('Accuracy:', accuracy)

# Print zipped clustered model size
zipped_size = get_gzipped_model(tflite_model_file)
print('Size of zipped clustered TFLite model: %.2f bytes' % zipped_size)
print('Size in KB: %.2f KB' % (zipped_size / 1024))


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Accuracy: 0.9175
Size of zipped clustered TFLite model: 128063.00 bytes
Size in KB: 125.06 KB


In [10]:
import numpy as np
import tensorflow as tf
import os
import time
import psutil
from pathlib import Path

# 1. Load TFLite model
tflite_model_file = 'tflite_models/model_clustered.tflite'
interpreter = tf.lite.Interpreter(model_path=tflite_model_file)
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

# 2. Evaluate accuracy + time
pred_list = []
start_time = time.time()

for image in X_test:
    input_data = np.array(image, dtype=np.float32).reshape(1, 28, 28, 1)
    interpreter.set_tensor(input_index, input_data)
    interpreter.invoke()
    prediction = interpreter.get_tensor(output_index)
    pred_list.append(np.argmax(prediction))

end_time = time.time()
total_time = end_time - start_time
avg_time_per_image = total_time / len(X_test)

# 3. Accuracy calculation
correct = sum(int(pred_list[i] == np.argmax(y_test[i])) for i in range(len(pred_list)))
accuracy = correct / len(pred_list)

# 4. Model file size in KB
model_size_kb = os.path.getsize(tflite_model_file) / 1024

# 5. Memory used during inference
process = psutil.Process(os.getpid())
memory_used_mb = process.memory_info().rss / (1024 * 1024)  # in MB

# 6. Print results
print(f" Accuracy: {accuracy * 100:.2f}%")
print(f" Model Size: {model_size_kb:.2f} KB")
print(f" Memory Usage: {memory_used_mb:.2f} MB")
print(f" Avg Inference Time per Image: {avg_time_per_image * 1000:.2f} ms")


print('Size of zipped clustered TFLite model: %.2f bytes' % zipped_size)
print('Size in KB: %.2f KB' % (zipped_size / 1024))


 Accuracy: 91.75%
 Model Size: 706.16 KB
 Memory Usage: 121.19 MB
 Avg Inference Time per Image: 0.16 ms
Size of zipped clustered TFLite model: 128063.00 bytes
Size in KB: 125.06 KB
